In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path = r'raw\1481069814_relax_challenge\relax_challenge\takehome_user_engagement.csv'
user_engagement_df = pd.read_csv(file_path)

user_engagement_df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


## Explore data

In [3]:
user_engagement_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [4]:
user_engagement_df.describe()

,user_id,visited
count,207917.000000,207917.0
mean,5913.314197,1.0
std,3394.941674,0.0
min,1.000000,1.0
25%,3087.000000,1.0
50%,5682.000000,1.0
75%,8944.000000,1.0
max,12000.000000,1.0


In [5]:
user_engagement_df['time_stamp'] = pd.to_datetime(user_engagement_df['time_stamp'])
user_engagement_df.set_index( 'time_stamp', inplace=True)

In [6]:
user_engagement_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 207917 entries, 2014-04-22 03:53:30 to 2014-01-26 08:57:12
Data columns (total 2 columns):
user_id    207917 non-null int64
visited    207917 non-null int64
dtypes: int64(2)
memory usage: 4.8 MB


Reference rolling techniques

https://stackoverflow.com/questions/19188217/using-rolling-functions-on-multi-index-dataframe-in-pandas

https://stackoverflow.com/questions/15771472/pandas-rolling-mean-by-time-interval

In [7]:
# Identify that a user is engaged if they login 3 days in a 7 day period
seven_day_rolling = user_engagement_df.groupby('user_id')['visited'].rolling('7d').count().reset_index()
engagement_mask = seven_day_rolling['visited'] >= 3

engaged_user_label = seven_day_rolling[engagement_mask]
engaged_user_label['engaged'] = 1

distinct_engaged_label = engaged_user_label[['user_id', 'engaged']].drop_duplicates()
distinct_engaged_label.head()

C:\Users\ggupta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,engaged
9,2,1
27,10,1
312,20,1
331,33,1
354,42,1


## Import and Explore the other user dataset

In [8]:
file_path = r'raw\1481069814_relax_challenge\relax_challenge\takehome_users.csv'
user_df = pd.read_csv(file_path, encoding='latin-1')

user_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [9]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [10]:
user_df.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [11]:
user_df.org_id.unique()

array([ 11,   1,  94, 193, 197,  37,  74, 302, 318,  69, 130, 254,   0,
       175, 211,   2,  58,  54,   7,   6, 123,  81,  56, 134,  92, 251,
       283,  98, 401,  33,  59, 109, 276, 208, 332,  64, 235,  63,  14,
         4, 127,  32, 224, 304,  61,  51, 229,  88, 325, 150, 347, 203,
        21, 409, 215, 141,   5, 107, 184, 310, 367, 389,  89, 242,  68,
        16,  49, 185,  91, 172,   8,  43, 249,  26, 139,  70,  10, 252,
        53, 119,   3, 196, 335, 375, 303, 307,  25, 370,  83,  27, 306,
       118, 295,  19, 301, 100, 112, 263,  39,  12,  86, 402,  42, 103,
       373, 268,  30, 406,  48, 313,   9, 138, 261,  35, 264,  95,  55,
        60, 234,  28, 410, 170, 374,  15,  17, 364,  62, 244, 342, 225,
       121, 255, 411, 217, 165, 415, 104, 329, 278, 116, 151, 290, 200,
       187, 122, 353, 135, 328,  36,  13, 341, 349, 198, 153, 108,  77,
       142, 275, 176,  87, 191, 102, 245, 154, 218, 385,  76, 321, 159,
       371,  50,  75, 333, 137, 292, 286, 131, 178, 213, 376, 36

## Feature engineering by calculating days since instead of just the date

In [12]:
user_df['todays_date'] = pd.to_datetime('30 May 2014')
user_df['days_since_creation'] = (user_df['todays_date'] - pd.to_datetime(user_df.creation_time)).dt.days
user_df['days_since_creation'].head()


0     37
1    195
2    436
3    373
4    497
Name: days_since_creation, dtype: int64

It is not clear what unit of 'days_since_last_session' column should be but 'second' seems to give the correct year

In [13]:
user_df['days_since_last_session'] = (user_df['todays_date'] - pd.to_datetime(user_df['last_session_creation_time'], unit='s') ).dt.days

user_df['days_since_last_session'].head()

0     37.0
1     59.0
2    436.0
3    372.0
4    492.0
Name: days_since_last_session, dtype: float64

In [14]:
label_enc = LabelEncoder()

creation_source_labels = label_enc.fit_transform(user_df.creation_source)

ohe_enc = OneHotEncoder(sparse=False)

creation_source_encoded = ohe_enc.fit_transform(creation_source_labels.reshape(-1,1))

creation_source_encoded

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])

### Missing values 
One possible way could be to replace missing values in days_since_last_session with a large value say 100000 days

In [15]:

user_df.days_since_last_session.head(10)

user_df.days_since_last_session.fillna(100000, inplace=True)
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 13 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
todays_date                   12000 non-null datetime64[ns]
days_since_creation           12000 non-null int64
days_since_last_session       12000 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(5), object(4)
memory usage: 1.2+ MB


Missing values are in invited_by_user_id

Address invited_by_user_id such that 
* invited = 0 if invited_by_user_id = NaN (i.e. user not invited)
* invited = 1 if invited_by_user_id = any integer (user invited)


In [16]:
user_df['invited'] = np.heaviside(user_df.invited_by_user_id,1)

user_df['invited'].fillna(0, inplace=True)
user_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,todays_date,days_since_creation,days_since_last_session,invited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,2014-05-30,37,37.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2014-05-30,195,59.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,2014-05-30,436,436.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,2014-05-30,373,372.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,2014-05-30,497,492.0,1.0


In [17]:
user_df.rename(columns={'object_id':'user_id' }, inplace=True)

In [18]:
# Join target label with features
user_and_target_df = pd.merge(user_df, distinct_engaged_label, on='user_id', how='left')
user_and_target_df.engaged.fillna(0, inplace=True)

user_and_target_df.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,todays_date,days_since_creation,days_since_last_session,invited,engaged
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,2014-05-30,37,37.0,1.0,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2014-05-30,195,59.0,1.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,2014-05-30,436,436.0,1.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,2014-05-30,373,372.0,1.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,2014-05-30,497,492.0,1.0,0.0


In [19]:
column_list = ['opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'days_since_creation', 
               'days_since_last_session', 'invited']

X = np.column_stack((creation_source_encoded, user_and_target_df[column_list]))
X

y = user_and_target_df['engaged']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


random_forest_clf = RandomForestClassifier(random_state=0)
random_forest_clf.fit(X_train, y_train)

print('Prediction on training data: ', random_forest_clf.score(X_train, y_train))

print('Prediction on test data: ', random_forest_clf.score(X_test, y_test))

Prediction on training data:  0.9964583333333333
Prediction on test data:  0.9654166666666667


In [21]:
random_forest_clf.feature_importances_

array([0.00305001, 0.00158229, 0.00301785, 0.00252229, 0.00187655,
       0.0036813 , 0.00338572, 0.06628147, 0.24085363, 0.67175886,
       0.00199004])

The model achieves reasonably accurate results with the random forest classifier.

'days_since_last_session', 'days_since_creation' and 'org_id' are the most important features. These features make sense in terms of their importance
               